In [ ]:
# -*-Encoding: utf-8 -*-
################################################################################
#
# Copyright (c) 2022 Baidu.com, Inc. All Rights Reserved
#
################################################################################
"""
Description: Training and Validation
Authors: Lu,Xinjiang (luxinjiang@baidu.com)
Date:    2022/03/10
"""
import os
import time
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from typing import Callable
from torch.utils.data import DataLoader
from common import EarlyStopping
from common import adjust_learning_rate
from common import Experiment
from common import traverse_wind_farm
from prepare import prep_env


def val(experiment, data_loader, criterion):
    # type: (Experiment, DataLoader, Callable) -> np.array
    """
    Desc:
        Validation function
    Args:
        experiment:
        data_loader:
        criterion:
    Returns:
        The validation loss
    """
    validation_loss = []
    for i, (batch_x, batch_y) in enumerate(data_loader):
        sample, true = experiment.process_one_batch(batch_x, batch_y)
        loss = criterion(sample, true)
        validation_loss.append(loss.item())
    validation_loss = np.average(validation_loss)
    return validation_loss


def train_and_val(experiment, model_folder, is_debug=False):
    # type: (Experiment, str, bool) -> None
    """
    Desc:
        Training and validation
    Args:
        experiment:
        model_folder: folder name of the model
        is_debug:
    Returns:
        None
    """
    args = experiment.get_args()
    model = experiment.get_model()
    train_data, train_loader = experiment.get_data(flag='train')
    val_data, val_loader = experiment.get_data(flag='val')

    path_to_model = os.path.join(args["checkpoints"], model_folder)
    if not os.path.exists(path_to_model):
        os.makedirs(path_to_model)

    time_now = time.time()
    early_stopping = EarlyStopping(patience=args["patience"], verbose=True)
    model_optim = experiment.get_optimizer()
    criterion = Experiment.get_criterion()

    epoch_start_time = time.time()
    model.cuda()
    for epoch in tqdm(range(args["train_epochs"])):
        iter_count = 0
        train_loss = []
        model.train()
        for i, (batch_x, batch_y) in enumerate(train_loader):
            # batch_x.cuda()
            # batch_y.cuda()
            iter_count += 1
            sample, truth = experiment.process_one_batch(batch_x, batch_y)
            loss = criterion(sample, truth)
            train_loss.append(loss.item())
            loss.backward()
            # model_optim.minimize(loss)
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=50.0, norm_type=2)
            model_optim.step()
        val_loss = val(experiment, val_loader, criterion)

        if is_debug:
            train_loss = np.average(train_loss)
            epoch_end_time = time.time()
            print("Epoch: {}, \nTrain Loss: {}, \nValidation Loss: {}".format(epoch, train_loss, val_loss))
            print("Elapsed time for epoch-{}: {}".format(epoch, epoch_end_time - epoch_start_time))
            epoch_start_time = epoch_end_time

        # Early Stopping if needed
        early_stopping(val_loss, model, path_to_model, args["turbine_id"])
        if early_stopping.early_stop:
            print("Early stopped! ")
            break
        adjust_learning_rate(model_optim, epoch + 1, args)

print('a')
# if __name__ == "__main__":
#     settings = prep_env()
#     #
#     # Set up the initial environment
#     # Current settings for the model
#     cur_setup = '{}_t{}_i{}_o{}_ls{}_train{}_val{}'.format(
#         settings["filename"], settings["task"], settings["input_len"], settings["output_len"], settings["lstm_layer"],
#         settings["train_size"], settings["val_size"]
#     )
#     traverse_wind_farm(train_and_val, settings, cur_setup)

settings = prep_env()
    #
    # Set up the initial environment
    # Current settings for the model
cur_setup = '{}_t{}_i{}_o{}_ls{}_train{}_val{}'.format(
        settings["filename"], settings["task"], settings["input_len"], settings["output_len"], settings["lstm_layer"],
        settings["train_size"], settings["val_size"]
    )
traverse_wind_farm(train_and_val, settings, cur_setup)

a
Experimental settings are: 
{'data_path': './data/', 'filename': 'wtbdata_245days.csv', 'task': 'MS', 'target': 'Patv', 'checkpoints': './checkpoints/', 'input_len': 144, 'output_len': 288, 'start_col': 3, 'in_var': 10, 'out_var': 1, 'day_len': 144, 'train_size': 153, 'val_size': 16, 'test_size': 15, 'total_size': 184, 'lstm_layer': 2, 'dropout': 0.05, 'num_workers': 5, 'train_epochs': 10, 'batch_size': 32, 'patience': 3, 'lr': 0.0001, 'lr_adjust': 'type1', 'capacity': 134, 'turbine_id': 0, 'pred_file': './predict.py', 'stride': 1, 'is_debug': False}
>>>>>>> Training Turbine   0 >>>>>>>>>>>>>>>>>>>>>>>>>>



/home/ma-user/work/common.py:197: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip = torch.nn.utils.clip_grad_norm(parameters=self.model.parameters(), max_norm=50.0)
 30%|███       | 3/10 [01:23<03:14, 27.84s/it]

Early stopped! 
>>>>>>> Training Turbine   1 >>>>>>>>>>>>>>>>>>>>>>>>>>




 70%|███████   | 7/10 [02:46<01:11, 23.72s/it]

Early stopped! 
>>>>>>> Training Turbine   2 >>>>>>>>>>>>>>>>>>>>>>>>>>



...

Early stopped! 
>>>>>>> Training Turbine 132 >>>>>>>>>>>>>>>>>>>>>>>>>>




 60%|██████    | 6/10 [02:30<01:40, 25.12s/it]

Early stopped! 
>>>>>>> Training Turbine 133 >>>>>>>>>>>>>>>>>>>>>>>>>>




 70%|███████   | 7/10 [02:49<01:12, 24.15s/it]

Early stopped! 
